SHAP

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00


In [18]:
import PyPDF2
# Open the PDF file

def open_pdf_text(file_path):
  with open(file_path, 'rb') as file:
      reader = PyPDF2.PdfReader(file)

      # Extract text from each page
      text_dict = dict()
      for page_num in range(len(reader.pages)):
          page = reader.pages[page_num]
          text_dict[page_num] = page.extract_text()
      return text_dict

text_dict_uber = open_pdf_text('Uber-Q1-24-Earnings-Press-Release.pdf')
text_dict_family = open_pdf_text('American-Families-and-Living-Arrangements-2022.pdf')

In [19]:
print(text_dict_uber[0])

 
 
 
1    
Uber Announces Results for First Quarter  2024  
 
Trips grew 21% year-over-year; MAPCs and monthly trips per MAPC grew 15% and 6% year-over-year, respectively  
Gross Bookings grew  20% year-over-year and 21% year-over-year on a constant currency basis  
Income from operations of $172 million; Adjusted EBITDA of $1.4 billion , up 82% year-over-year 
Operating cash flow of $1.4 billion ; Free cash flow of $1.4 billion  
 
SAN FRANCISCO – May  8, 2024  – Uber Technologies, Inc. (NYSE: UBER) today announced financial results for the quarter  ended 
March  31, 2024 . 
 
“Our results this quarter once again demonstrate our ability to deliver consistent, profitable growth at scale,” said Dara 
Khosrowshahi, CEO. “More than 7 million  people now choose to earn flexibly on Uber every month, with driver earnings of $16.6 
billion  continuing to grow faster than our topline .” 
 
“Our multi -year growth framework is on track, with audience up 15% and frequency up 6% in Q1,” said Pra

In [20]:
print(text_dict_family[0])

America’s Families and  
Living Arrangements: 2022 
By Paul F. Hemez, Chanell N. Washington, and Rose M. KreiderCurrent Population ReportsPopulation Characteristics
Issued May 2024P20-587
INTRODUCTION 
This report provides a demographic profile of the 
households and living arrangements of Americans, and how these have changed over time. Additionally, the COVID-19 pandemic affected the economic well-being of millions of families, as many businesses oper-
ated under reduced hours or were lost, causing many 
people to become underemployed or unemployed. Thus, a secondary goal of this report is to examine changes in the economic well-being of American families before and after the start of the COVID-19 pandemic.
This report uses data from the American Community 
Survey (ACS) and the Current Population Survey’s 
Annual Social and Economic Supplement (CPS ASEC).
1  
It capitalizes on the strengths of both datasets, using ACS data about how basic family and household char-acteristics vary ac

In [16]:
def get_page(page_num, name):
  text_dict_name = f'text_dict_{name}'  # Concatenate 'text_dict_' with the provided name
  text_dict = globals().get(text_dict_name)  # Access the variable using globals()

  if text_dict is None:
    raise ValueError(f"Dictionary with name {text_dict_name} not found")
  return text_dict[page_num]

get_page(2, 'family')

'U.S. Census Bureau  3\nTable 1.\nHouseholds by Type and Selected Characteristics: 2019\n(Numbers in thousands)\nCharacteristic\nAll  \nhouseholdsFamily households Nonfamily households\nTotal Married \ncoupleOther families\nTotal Male  \nhouseholderFemale \nhouseholderMale  \nhouseholderFemale  \nhouseholder\nAll Households . . . . . . . . . . . . . . . . . . 122,800 79,590 58,370 6,168 15,060 43,210 20,330 22,880\nAge of Householder\n15 to 24 years  .................... 4,415 1,648 675 370 604 2,766 1,405 1,361\n25 to 34 years  .................... 18,580 11,440 7,387 1,235 2,822 7,137 4,139 2,998\n35 to 44 years  .................... 20,990 16,470 11,520 1,407 3,551 4,511 2,732 1,779\n45 to 54 years  .................... 21,840 16,390 11,950 1,323 3,115 5,446 2,996 2,449\n55 to 64 years  .................... 23,990 15,670 12,410 977 2,278 8,321 3,958 4,363\n65 years and over  ................ 33,000 17,970 14,430 856 2,685 15,030 5,104 9,925\nRace and Hispanic Origin of \nHouseholder